In [1]:
import sys
from pathlib import Path
import os

lib_path = Path("/workspaces/data-scientist-at-magenta/src/code_location_interview/code_location_interview/assets/lib")

# Add it to sys.path if it's not already there
if str(lib_path) not in sys.path:
    sys.path.append(str(lib_path))

from datawarehouse import create

In [2]:
duckdb_database = os.environ.get("DUCKDB_DATABASE")

# Create

In [5]:
create(duckdb_database)

Initializing data warehouse at: /workspaces/data-scientist-at-magenta/src/code_location_interview/code_location_interview/dwh/database.duckdb
Creating raw data tables...
Creating feature tables...
Creating model registry tables...
Creating prediction tables...

Successfully created 8 tables:
  - feature_statistics
  - model_registry
  - predictions
  - processed_features
  - raw_core_data
  - raw_customer_interactions
  - raw_labels
  - raw_usage_info

Data warehouse initialization completed successfully!
Database file: /workspaces/data-scientist-at-magenta/src/code_location_interview/code_location_interview/dwh/database.duckdb


# Check

In [22]:
conn.execute("DROP TABLE IF EXISTS predictions")

In [3]:
import duckdb

conn = duckdb.connect(database=duckdb_database)
tables = conn.execute("SHOW TABLES").fetchdf()
print(tables)

                        name
0         feature_statistics
1             model_registry
2                predictions
3         processed_features
4              raw_core_data
5  raw_customer_interactions
6                 raw_labels
7             raw_usage_info


# Model

In [9]:
conn.execute("SELECT * FROM model_registry").fetchdf()

,model_id,model_name,version,model_artifact,metadata,status,is_current_model,created_at
0,model_001,RandomForest-General,1.0,"[128, 4, 149, 232, 3, 1, 0, 0, 0, 0, 0, 140, 2...","{""algorithm"": ""RandomForestClassifier"", ""hyper...",deployed,True,2025-07-02 23:00:09.682


In [5]:
query = """
    SELECT 
        model_id,
        model_name,
        version,
        model_artifact,
        metadata,
        created_at
    FROM model_registry
    WHERE is_current_model = TRUE AND status = 'deployed'
    ORDER BY created_at DESC
    LIMIT 1
"""

In [6]:
conn.execute(query).fetchdf()

,model_id,model_name,version,model_artifact,metadata,created_at
0,model_001,RandomForest-General,1.0,"[128, 4, 149, 232, 3, 1, 0, 0, 0, 0, 0, 140, 2...","{""algorithm"": ""RandomForestClassifier"", ""hyper...",2025-07-02 23:00:09.682


In [8]:
import pickle
from pathlib import Path
import json
import duckdb

conn = duckdb.connect(database=duckdb_database)
# tables = conn.execute("SHOW TABLES").fetchdf()
# print(tables)

artifact_path = Path("../src/code_location_interview/code_location_interview/assets/artifacts/initial_model.pkl")
# Load the pickled model artifact as bytes
with open(artifact_path, "rb") as f:
    model_artifact = f.read()

model = pickle.loads(model_artifact)

# Define model details
model_id = "model_001"
model_name = "RandomForest-General"
version = "1.0"

status = "deployed"
is_current_model = True
metadata = {
    "algorithm": "RandomForestClassifier",
    "hyperparameters": model.get_params()
}

# Load the pickled model artifact as bytes
with open(artifact_path, "rb") as f:
    model_artifact = f.read()

# Insert the row into model_registry
conn.execute("""
    INSERT INTO model_registry (
        model_id, model_name, version, model_artifact, metadata, status, is_current_model
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
""", [
    model_id,
    model_name,
    version,
    model_artifact,
    json.dumps(metadata), 
    status,
    is_current_model
])
